In [4]:
!pip install -q groq
!pip install gradio

   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
    --------------------------------------- 0.2/12.6 MB 5.6 MB/s eta 0:00:03
   - -------------------------------------- 0.4/12.6 MB 5.3 MB/s eta 0:00:03
   - -------------------------------------- 0.6/12.6 MB 4.8 MB/s eta 0:00:03
   -- ------------------------------------- 0.9/12.6 MB 5.1 MB/s eta 0:00:03
   --- ------------------------------------ 1.1/12.6 MB 5.2 MB/s eta 0:00:03
   ---- ----------------------------------- 1.4/12.6 MB 5.1 MB/s eta 0:00:03
   ---- ----------------------------------- 1.5/12.6 MB 4.9 MB/s eta 0:00:03
   ----- ---------------------------------- 1.8/12.6 MB 4.9 MB/s eta 0:00:03
   ------ --------------------------------- 2.0/12.6 MB 4.9 MB/s eta 0:00:03
   ------- -------------------------------- 2.3/12.6 MB 5.0 MB/s eta 0:00:03
   ------- -------------------------------- 2.5/12.6 MB 4.9 MB/s eta 0:00:03
   -------- ------------------------------- 2.7/12.6 MB 4.9 MB/s eta 0:00:03
   ---

In [5]:
import gradio as gr
from groq import Groq

# Replace with your actual Groq API key
GROQ_API_KEY = ''

# Initialize the Groq client
client = Groq(api_key=GROQ_API_KEY)

def classify_text_with_llama3(text, prompt_type):
    """
    Uses Llama3 via Groq to classify the text based on the given prompt type.
    """
    response = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": f"{prompt_type}:\n\n{text}",
            }
        ],
        model="llama3-70b-8192"
    )
    
    return response.choices[0].message.content.strip()

def chunk_text_with_llama3(input_text):
    """
    Chunks the input text into code and non-code blocks using Llama3 for classification.
    """
    code_chunks = []
    non_code_chunks = []
    current_chunk = []
    current_type = None

    # Split text into lines
    lines = input_text.splitlines()

    for line in lines:
        # Determine whether the line is code or non-code
        classification = classify_text_with_llama3(line, "Classify the following line as 'Code' or 'Non-Code'. Provide a single word answer.")
        
        # Determine the type for the first line
        if current_type is None:
            current_type = classification.lower()

        # Check if current line matches the current type
        if classification.lower() == current_type:
            current_chunk.append(line)
        else:
            if current_type == 'code':
                code_chunks.append('\n'.join(current_chunk))
            else:
                non_code_chunks.append('\n'.join(current_chunk))
            current_chunk = [line]
            current_type = classification.lower()

    # Append the last chunk
    if current_chunk:
        if current_type == 'code':
            code_chunks.append('\n'.join(current_chunk))
        else:
            non_code_chunks.append('\n'.join(current_chunk))

    return code_chunks, non_code_chunks

def classify_and_process(input_text):
    """
    Processes the input text by chunking it into code and non-code blocks and determines overall input status.
    """
    code_chunks, non_code_chunks = chunk_text_with_llama3(input_text)

    # Classify entire input to determine overall status using Llama3
    overall_status = classify_text_with_llama3(input_text, "Classify the entire input text as 'Code' or 'Non-Code'. Provide a single word answer.")

    return {
        "non_code": non_code_chunks,
        "code": code_chunks,
        "overall_status": overall_status
    }

def format_output(chunks):
    """
    Formats each chunk with proper indentation.
    """
    return "\n".join(f"        {line.strip()}" for chunk in chunks for line in chunk.splitlines() if line.strip())

def process_text(input_text):
    """
    Process the text using the classification functions and return formatted results.
    """
    output = classify_and_process(input_text)
    formatted_non_code = format_output(output["non_code"])
    formatted_code = format_output(output["code"])
    overall_status = output["overall_status"]
    
    return f"NON CODE CHUNKS:\n{formatted_non_code}\n\nCODE CHUNKS:\n{formatted_code}\n\nOVERALL INPUT STATUS : {(overall_status).upper()}"

# Create Gradio interface
iface = gr.Interface(
    fn=process_text,
    inputs=gr.Textbox(lines=10, placeholder="Enter text here..."),
    outputs=gr.Textbox(),
    title="Code vs Non-Code Classifier",
    description="Classify and chunk text into code and non-code parts using Llama3 via Groq."
)

# Launch the interface
iface.launch()


C:\Users\madhu\anaconda3\envs\cuda_test\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [6]:
print("Hey")

Hey
